## Install Pyspark

In [1]:
!pip install pyspark py4j

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Importing Libraries

In [2]:
import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark import SparkContext, SparkConf
from pyspark.sql import *
from pyspark.sql.functions import udf
from pyspark.sql.types import *

 
import pandas as pd
#from pyspark.sql import Row

##Build Session

In [3]:
spark = SparkSession.builder \
    .appName("zillow") \
    .master("local[*]") \
    .config("spark.sql.shuffle.partitions", 8) \
    .config("spark.driver.memory", "4g") \
    .config("spark.executor.memory", "2g") \
    .getOrCreate()

In [4]:
spark

## Import Data

In [5]:
df = spark.read.csv("/content/zillow.csv", header=True)

In [6]:
df.show()

+--------------+-------+------------+-----+-----------+----------+--------------------+--------------------+--------------------+
|         title|address|        city|state|postal_code|     price|  facts and features|real estate provider|                 url|
+--------------+-------+------------+-----+-----------+----------+--------------------+--------------------+--------------------+
|Condo for sale|   null|  Somerville|   MA|      02145|  $342,000|2 bds, 1.0 ba ,70...|William Raveis R....|https://www.zillo...|
|Condo for sale|   null|      Boston|   MA|      02116|$1,700,000|2 bds, 2.0 ba ,12...|Century 21 North ...|https://www.zillo...|
|Condo for sale|   null|      Boston|   MA|      02118|  $336,500|1 bds, 1.0 ba ,10...|Maloney Propertie...|https://www.zillo...|
|House for sale|   null|      Boston|   MA|      02118|$9,950,000|4 bds, 7.0 ba ,68...|Campion & Company...|https://www.zillo...|
|Condo for sale|   null|      Boston|   MA|      02128|  $479,000|2 bds, 3.0 ba ,10...|Ber

We notice that addreses are null and after futher ivestigating the csv it is confirmed that the address column is an empty column. Also all properties are from the MA area meaning the state column is redudant. 

In [7]:
df = df.drop("address")
df = df.drop("state")
df.show()

+--------------+------------+-----------+----------+--------------------+--------------------+--------------------+
|         title|        city|postal_code|     price|  facts and features|real estate provider|                 url|
+--------------+------------+-----------+----------+--------------------+--------------------+--------------------+
|Condo for sale|  Somerville|      02145|  $342,000|2 bds, 1.0 ba ,70...|William Raveis R....|https://www.zillo...|
|Condo for sale|      Boston|      02116|$1,700,000|2 bds, 2.0 ba ,12...|Century 21 North ...|https://www.zillo...|
|Condo for sale|      Boston|      02118|  $336,500|1 bds, 1.0 ba ,10...|Maloney Propertie...|https://www.zillo...|
|House for sale|      Boston|      02118|$9,950,000|4 bds, 7.0 ba ,68...|Campion & Company...|https://www.zillo...|
|Condo for sale|      Boston|      02128|  $479,000|2 bds, 3.0 ba ,10...|Berkshire Hathawa...|https://www.zillo...|
|House for sale| East Boston|      02128|  $899,000|3 bds, 3.0 ba ,23...

## Question 1 (No of Beadrooms)

In [8]:
from pyspark.sql.functions import pandas_udf
import pyspark.sql.functions as F

# Define a python function that takes a string as an input
def split_and_convert(string):
  first_element = string.split()[0]
  # Try to convert the first element to an integer
  try:
    first_int = int(first_element)
  except ValueError:
    first_int = 0
  return first_int
# Register the python function as a pyspark UDF with an integer return type
split_and_convert_udf = F.udf(split_and_convert, IntegerType())

In [9]:
# Apply the UDF to a pyspark dataframe column and return a new column
beadR = split_and_convert_udf(df["facts and features"])

# Add the new column to the dataframe
df = df.withColumn("beadR", split_and_convert_udf(df["facts and features"]))
# Show the first 10 rows of the dataframe with the new column
df.show(10)

+--------------+------------+-----------+----------+--------------------+--------------------+--------------------+-----+
|         title|        city|postal_code|     price|  facts and features|real estate provider|                 url|beadR|
+--------------+------------+-----------+----------+--------------------+--------------------+--------------------+-----+
|Condo for sale|  Somerville|      02145|  $342,000|2 bds, 1.0 ba ,70...|William Raveis R....|https://www.zillo...|    2|
|Condo for sale|      Boston|      02116|$1,700,000|2 bds, 2.0 ba ,12...|Century 21 North ...|https://www.zillo...|    2|
|Condo for sale|      Boston|      02118|  $336,500|1 bds, 1.0 ba ,10...|Maloney Propertie...|https://www.zillo...|    1|
|House for sale|      Boston|      02118|$9,950,000|4 bds, 7.0 ba ,68...|Campion & Company...|https://www.zillo...|    4|
|Condo for sale|      Boston|      02128|  $479,000|2 bds, 3.0 ba ,10...|Berkshire Hathawa...|https://www.zillo...|    2|
|House for sale| East Bo

## Question 2 (No of bathrooms)

In [10]:
# Import the pyspark.sql.functions module
import pyspark.sql.functions as F
# Define a python function that takes a string as an input
def split_and_convert(string):
  # Split the string by commas and take the second, third and fourth digits after the first comma

  digit = (string.split(",")[1]).split()[0]
  if (digit=="None"):
    digit = 0.0
  else:
   digit = float(digit)

  # Try to convert the digits to a float number
  try:
    float_number = digit

  # If it fails, use a default value of zero
  except ValueError:
    float_number = 0.0
  # Return the float number
  return float_number
# Register the python function as a pyspark UDF with a float return type
split_and_convert_udf = F.udf(split_and_convert, FloatType())

In [11]:
# Apply the UDF to a pyspark dataframe column and return a new column
bathR = split_and_convert_udf(df["facts and features"])

df = df.withColumn("bathR", split_and_convert_udf(df["facts and features"]))
df.show(10)

+--------------+------------+-----------+----------+--------------------+--------------------+--------------------+-----+-----+
|         title|        city|postal_code|     price|  facts and features|real estate provider|                 url|beadR|bathR|
+--------------+------------+-----------+----------+--------------------+--------------------+--------------------+-----+-----+
|Condo for sale|  Somerville|      02145|  $342,000|2 bds, 1.0 ba ,70...|William Raveis R....|https://www.zillo...|    2|  1.0|
|Condo for sale|      Boston|      02116|$1,700,000|2 bds, 2.0 ba ,12...|Century 21 North ...|https://www.zillo...|    2|  2.0|
|Condo for sale|      Boston|      02118|  $336,500|1 bds, 1.0 ba ,10...|Maloney Propertie...|https://www.zillo...|    1|  1.0|
|House for sale|      Boston|      02118|$9,950,000|4 bds, 7.0 ba ,68...|Campion & Company...|https://www.zillo...|    4|  7.0|
|Condo for sale|      Boston|      02128|  $479,000|2 bds, 3.0 ba ,10...|Berkshire Hathawa...|https://ww

## Question 3 (Sqft)

In [12]:
def split_and_convert(string):
  digit = (string.split(",")[2]).split()[0]

  if (digit != "None"):
    sqft = int(digit)
  else:
    sqft = 0

  return sqft
# Register the python function as a pyspark UDF with a float return type
split_and_convert_udf = F.udf(split_and_convert, IntegerType())

In [13]:
# Apply the UDF to a pyspark dataframe column and return a new column
sqft = split_and_convert_udf(df["facts and features"])

df = df.withColumn("sqft", split_and_convert_udf(df["facts and features"]))
df.show(10)

+--------------+------------+-----------+----------+--------------------+--------------------+--------------------+-----+-----+----+
|         title|        city|postal_code|     price|  facts and features|real estate provider|                 url|beadR|bathR|sqft|
+--------------+------------+-----------+----------+--------------------+--------------------+--------------------+-----+-----+----+
|Condo for sale|  Somerville|      02145|  $342,000|2 bds, 1.0 ba ,70...|William Raveis R....|https://www.zillo...|    2|  1.0| 705|
|Condo for sale|      Boston|      02116|$1,700,000|2 bds, 2.0 ba ,12...|Century 21 North ...|https://www.zillo...|    2|  2.0|1228|
|Condo for sale|      Boston|      02118|  $336,500|1 bds, 1.0 ba ,10...|Maloney Propertie...|https://www.zillo...|    1|  1.0|1000|
|House for sale|      Boston|      02118|$9,950,000|4 bds, 7.0 ba ,68...|Campion & Company...|https://www.zillo...|    4|  7.0|6836|
|Condo for sale|      Boston|      02128|  $479,000|2 bds, 3.0 ba ,10

At this point i would say that the column facts and features have become redundant

In [14]:
df = df.drop("facts and features")
df.show(10)

+--------------+------------+-----------+----------+--------------------+--------------------+-----+-----+----+
|         title|        city|postal_code|     price|real estate provider|                 url|beadR|bathR|sqft|
+--------------+------------+-----------+----------+--------------------+--------------------+-----+-----+----+
|Condo for sale|  Somerville|      02145|  $342,000|William Raveis R....|https://www.zillo...|    2|  1.0| 705|
|Condo for sale|      Boston|      02116|$1,700,000|Century 21 North ...|https://www.zillo...|    2|  2.0|1228|
|Condo for sale|      Boston|      02118|  $336,500|Maloney Propertie...|https://www.zillo...|    1|  1.0|1000|
|House for sale|      Boston|      02118|$9,950,000|Campion & Company...|https://www.zillo...|    4|  7.0|6836|
|Condo for sale|      Boston|      02128|  $479,000|Berkshire Hathawa...|https://www.zillo...|    2|  3.0|1000|
|House for sale| East Boston|      02128|  $899,000|Berkshire Hathawa...|https://www.zillo...|    3|  3.

## Question 4 (Type of Listing)

In [15]:
def split_and_convert(string):
  propertyType = string.split()[0]
  if (propertyType == "Condo"):
    propertyType = "condo"
  elif (propertyType == "House"):
    propertyType = "house"
  elif (propertyType == "Multi-family"):
    propertyType = "multi-family home"
  elif (propertyType == "New"):
    propertyType = "new construction"  
  elif (propertyType == "Lot"):
    propertyType = "land"
  else:
    propertyType = "uknown"

  return propertyType

# Register the python function as a pyspark UDF with a float return type
split_and_convert_udf = F.udf(split_and_convert, StringType())

In [16]:
# Apply the UDF to a pyspark dataframe column and return a new column
propType = split_and_convert_udf(df["title"])

df = df.withColumn("propType", split_and_convert_udf(df["title"]))
df.show(10)

+--------------+------------+-----------+----------+--------------------+--------------------+-----+-----+----+--------+
|         title|        city|postal_code|     price|real estate provider|                 url|beadR|bathR|sqft|propType|
+--------------+------------+-----------+----------+--------------------+--------------------+-----+-----+----+--------+
|Condo for sale|  Somerville|      02145|  $342,000|William Raveis R....|https://www.zillo...|    2|  1.0| 705|   condo|
|Condo for sale|      Boston|      02116|$1,700,000|Century 21 North ...|https://www.zillo...|    2|  2.0|1228|   condo|
|Condo for sale|      Boston|      02118|  $336,500|Maloney Propertie...|https://www.zillo...|    1|  1.0|1000|   condo|
|House for sale|      Boston|      02118|$9,950,000|Campion & Company...|https://www.zillo...|    4|  7.0|6836|   house|
|Condo for sale|      Boston|      02128|  $479,000|Berkshire Hathawa...|https://www.zillo...|    2|  3.0|1000|   condo|
|House for sale| East Boston|   

## Question 5 (Type of offer)

In [17]:
def split_and_convert(string):
  exchange = (string.split()[-1]).lower()
  return exchange

split_and_convert_udf = F.udf(split_and_convert, StringType())

In [18]:
# Apply the UDF to a pyspark dataframe column and return a new column
offer = split_and_convert_udf(df["title"])

# Add the new column to the dataframe
df = df.withColumn("offer", split_and_convert_udf(df["title"]))

df.show(10)

+--------------+------------+-----------+----------+--------------------+--------------------+-----+-----+----+--------+-----+
|         title|        city|postal_code|     price|real estate provider|                 url|beadR|bathR|sqft|propType|offer|
+--------------+------------+-----------+----------+--------------------+--------------------+-----+-----+----+--------+-----+
|Condo for sale|  Somerville|      02145|  $342,000|William Raveis R....|https://www.zillo...|    2|  1.0| 705|   condo| sale|
|Condo for sale|      Boston|      02116|$1,700,000|Century 21 North ...|https://www.zillo...|    2|  2.0|1228|   condo| sale|
|Condo for sale|      Boston|      02118|  $336,500|Maloney Propertie...|https://www.zillo...|    1|  1.0|1000|   condo| sale|
|House for sale|      Boston|      02118|$9,950,000|Campion & Company...|https://www.zillo...|    4|  7.0|6836|   house| sale|
|Condo for sale|      Boston|      02128|  $479,000|Berkshire Hathawa...|https://www.zillo...|    2|  3.0|1000|

In [19]:
def check_offer(offer):
  return offer == "sale"

# Register the UDF as a Spark SQL function
check_offer_udf = udf(check_offer, BooleanType())

# Apply the UDF to the offer column and filter the rows where the UDF returns True
test = df.filter(check_offer_udf(df.offer))

test.show(10)

+--------------+------------+-----------+----------+--------------------+--------------------+-----+-----+----+--------+-----+
|         title|        city|postal_code|     price|real estate provider|                 url|beadR|bathR|sqft|propType|offer|
+--------------+------------+-----------+----------+--------------------+--------------------+-----+-----+----+--------+-----+
|Condo for sale|  Somerville|      02145|  $342,000|William Raveis R....|https://www.zillo...|    2|  1.0| 705|   condo| sale|
|Condo for sale|      Boston|      02116|$1,700,000|Century 21 North ...|https://www.zillo...|    2|  2.0|1228|   condo| sale|
|Condo for sale|      Boston|      02118|  $336,500|Maloney Propertie...|https://www.zillo...|    1|  1.0|1000|   condo| sale|
|House for sale|      Boston|      02118|$9,950,000|Campion & Company...|https://www.zillo...|    4|  7.0|6836|   house| sale|
|Condo for sale|      Boston|      02128|  $479,000|Berkshire Hathawa...|https://www.zillo...|    2|  3.0|1000|

In [20]:
def check_offer(offer):
  return offer == "rent"

# Register the UDF as a Spark SQL function
check_offer_udf = udf(check_offer, BooleanType())

# Apply the UDF to the offer column and filter the rows where the UDF returns True
test = df.filter(check_offer_udf(df.offer))

test.show()

+-----+----+-----------+-----+--------------------+---+-----+-----+----+--------+-----+
|title|city|postal_code|price|real estate provider|url|beadR|bathR|sqft|propType|offer|
+-----+----+-----------+-----+--------------------+---+-----+-----+----+--------+-----+
+-----+----+-----------+-----+--------------------+---+-----+-----+----+--------+-----+



In [21]:
def check_offer(offer):
  return offer == "sold"

# Register the UDF as a Spark SQL function
check_offer_udf = udf(check_offer, BooleanType())

# Apply the UDF to the offer column and filter the rows where the UDF returns True
test = df.filter(check_offer_udf(df.offer))

test.show()

+-----+----+-----------+-----+--------------------+---+-----+-----+----+--------+-----+
|title|city|postal_code|price|real estate provider|url|beadR|bathR|sqft|propType|offer|
+-----+----+-----------+-----+--------------------+---+-----+-----+----+--------+-----+
+-----+----+-----------+-----+--------------------+---+-----+-----+----+--------+-----+



In [22]:
def check_offer(offer):
  return offer == "foreclosure"

# Register the UDF as a Spark SQL function
check_offer_udf = udf(check_offer, BooleanType())

# Apply the UDF to the offer column and filter the rows where the UDF returns True
test = df.filter(check_offer_udf(df.offer))

test.show()

+-----------+----------+-----------+--------+--------------------+--------------------+-----+-----+----+--------+-----------+
|      title|      city|postal_code|   price|real estate provider|                 url|beadR|bathR|sqft|propType|      offer|
+-----------+----------+-----------+--------+--------------------+--------------------+-----+-----+----+--------+-----------+
|Foreclosure| Hyde Park|      02136|$289,900|Mass Realty Advisors|https://www.zillo...|    2|  1.0| 848|  uknown|foreclosure|
|Foreclosure|    Boston|      02124|$999,999|Landmark Unlimite...|https://www.zillo...|    7|  4.0|3030|  uknown|foreclosure|
|Foreclosure|Roslindale|      02131|$234,900|Century 21 North ...|https://www.zillo...|    1|  1.0| 624|  uknown|foreclosure|
|Foreclosure| Hyde Park|      02136|$289,900|Mass Realty Advisors|https://www.zillo...|    2|  1.0| 848|  uknown|foreclosure|
|Foreclosure| Hyde Park|      02136|$289,900|Mass Realty Advisors|https://www.zillo...|    2|  1.0| 848|  uknown|forec

## Question 6 (Filter entries that are not for sale)

In [23]:
df_sale = df.select("*")
df_sale = df.filter(df_sale.offer == "sale")

#Return listing with offer==forclosure using the check_offer_udf() function
test = df_sale.filter(check_offer_udf(df_sale.offer))

test.show()

+-----+----+-----------+-----+--------------------+---+-----+-----+----+--------+-----+
|title|city|postal_code|price|real estate provider|url|beadR|bathR|sqft|propType|offer|
+-----+----+-----------+-----+--------------------+---+-----+-----+----+--------+-----+
+-----+----+-----------+-----+--------------------+---+-----+-----+----+--------+-----+



In [24]:
df_sale.show()

+--------------+------------+-----------+----------+--------------------+--------------------+-----+-----+----+--------+-----+
|         title|        city|postal_code|     price|real estate provider|                 url|beadR|bathR|sqft|propType|offer|
+--------------+------------+-----------+----------+--------------------+--------------------+-----+-----+----+--------+-----+
|Condo for sale|  Somerville|      02145|  $342,000|William Raveis R....|https://www.zillo...|    2|  1.0| 705|   condo| sale|
|Condo for sale|      Boston|      02116|$1,700,000|Century 21 North ...|https://www.zillo...|    2|  2.0|1228|   condo| sale|
|Condo for sale|      Boston|      02118|  $336,500|Maloney Propertie...|https://www.zillo...|    1|  1.0|1000|   condo| sale|
|House for sale|      Boston|      02118|$9,950,000|Campion & Company...|https://www.zillo...|    4|  7.0|6836|   house| sale|
|Condo for sale|      Boston|      02128|  $479,000|Berkshire Hathawa...|https://www.zillo...|    2|  3.0|1000|

## Question 7 (Filter entries that are not for sale)

In [25]:
from pyspark.sql.functions import pandas_udf
import pyspark.sql.functions as F

def pPrice(string):
  num = ((string[1:]).split(","))
  if num[-1][-1]=="+":
    num[-1] = num[-1][:-1]
  num = ''.join(num)
  num = int(num)    
  return num

pPrice_udf = F.udf(pPrice, IntegerType())

In [26]:
# Apply the UDF to a pyspark dataframe column and return a new column
price = pPrice_udf(df_sale["price"])

# Add the new column to the dataframe
df_sale = df_sale.withColumn("price", pPrice_udf(df_sale["price"]))
# Show the first 10 rows of the dataframe with the new column
df_sale.show(10)

+--------------+------------+-----------+-------+--------------------+--------------------+-----+-----+----+--------+-----+
|         title|        city|postal_code|  price|real estate provider|                 url|beadR|bathR|sqft|propType|offer|
+--------------+------------+-----------+-------+--------------------+--------------------+-----+-----+----+--------+-----+
|Condo for sale|  Somerville|      02145| 342000|William Raveis R....|https://www.zillo...|    2|  1.0| 705|   condo| sale|
|Condo for sale|      Boston|      02116|1700000|Century 21 North ...|https://www.zillo...|    2|  2.0|1228|   condo| sale|
|Condo for sale|      Boston|      02118| 336500|Maloney Propertie...|https://www.zillo...|    1|  1.0|1000|   condo| sale|
|House for sale|      Boston|      02118|9950000|Campion & Company...|https://www.zillo...|    4|  7.0|6836|   house| sale|
|Condo for sale|      Boston|      02128| 479000|Berkshire Hathawa...|https://www.zillo...|    2|  3.0|1000|   condo| sale|
|House f

This is the answer, but i dislike it visualy so i move the columns around a bit

In [27]:
df_sale = df_sale.select("title","city","postal_code","real estate provider","url","offer","propType","beadR","bathR","sqft","price")
df_sale.show(10)

+--------------+------------+-----------+--------------------+--------------------+-----+--------+-----+-----+----+-------+
|         title|        city|postal_code|real estate provider|                 url|offer|propType|beadR|bathR|sqft|  price|
+--------------+------------+-----------+--------------------+--------------------+-----+--------+-----+-----+----+-------+
|Condo for sale|  Somerville|      02145|William Raveis R....|https://www.zillo...| sale|   condo|    2|  1.0| 705| 342000|
|Condo for sale|      Boston|      02116|Century 21 North ...|https://www.zillo...| sale|   condo|    2|  2.0|1228|1700000|
|Condo for sale|      Boston|      02118|Maloney Propertie...|https://www.zillo...| sale|   condo|    1|  1.0|1000| 336500|
|House for sale|      Boston|      02118|Campion & Company...|https://www.zillo...| sale|   house|    4|  7.0|6836|9950000|
|Condo for sale|      Boston|      02128|Berkshire Hathawa...|https://www.zillo...| sale|   condo|    2|  3.0|1000| 479000|
|House f

Much better

## Question 8 (Filter >10 beadrooms)

In [28]:
df_sale = df_sale.filter(df_sale.beadR <= 10)
df_sale.show(10)

+--------------+------------+-----------+--------------------+--------------------+-----+--------+-----+-----+----+-------+
|         title|        city|postal_code|real estate provider|                 url|offer|propType|beadR|bathR|sqft|  price|
+--------------+------------+-----------+--------------------+--------------------+-----+--------+-----+-----+----+-------+
|Condo for sale|  Somerville|      02145|William Raveis R....|https://www.zillo...| sale|   condo|    2|  1.0| 705| 342000|
|Condo for sale|      Boston|      02116|Century 21 North ...|https://www.zillo...| sale|   condo|    2|  2.0|1228|1700000|
|Condo for sale|      Boston|      02118|Maloney Propertie...|https://www.zillo...| sale|   condo|    1|  1.0|1000| 336500|
|House for sale|      Boston|      02118|Campion & Company...|https://www.zillo...| sale|   house|    4|  7.0|6836|9950000|
|Condo for sale|      Boston|      02128|Berkshire Hathawa...|https://www.zillo...| sale|   condo|    2|  3.0|1000| 479000|
|House f

In [29]:
def check_beads(beadR):
  return beadR > 10

check_beads_udf = udf(check_beads, BooleanType())
test = df_sale.filter(check_beads_udf(df_sale.beadR))

test.show()

+-----+----+-----------+--------------------+---+-----+--------+-----+-----+----+-----+
|title|city|postal_code|real estate provider|url|offer|propType|beadR|bathR|sqft|price|
+-----+----+-----------+--------------------+---+-----+--------+-----+-----+----+-----+
+-----+----+-----------+--------------------+---+-----+--------+-----+-----+----+-----+



## Question 9 (Filter by price)

In [30]:
df_sale = df_sale.filter(df_sale.price < 20000000)
df_sale = df_sale.filter(df_sale.price > 100000)
df_sale.show(10)

+--------------+------------+-----------+--------------------+--------------------+-----+--------+-----+-----+----+-------+
|         title|        city|postal_code|real estate provider|                 url|offer|propType|beadR|bathR|sqft|  price|
+--------------+------------+-----------+--------------------+--------------------+-----+--------+-----+-----+----+-------+
|Condo for sale|  Somerville|      02145|William Raveis R....|https://www.zillo...| sale|   condo|    2|  1.0| 705| 342000|
|Condo for sale|      Boston|      02116|Century 21 North ...|https://www.zillo...| sale|   condo|    2|  2.0|1228|1700000|
|Condo for sale|      Boston|      02118|Maloney Propertie...|https://www.zillo...| sale|   condo|    1|  1.0|1000| 336500|
|House for sale|      Boston|      02118|Campion & Company...|https://www.zillo...| sale|   house|    4|  7.0|6836|9950000|
|Condo for sale|      Boston|      02128|Berkshire Hathawa...|https://www.zillo...| sale|   condo|    2|  3.0|1000| 479000|
|House f

In [31]:
def check_price(price):
  return ((price > 20000000) or (price < 100000))

check_price_udf = udf(check_price, BooleanType())
test = df_sale.filter(check_price_udf(df_sale.price))

test.show()

+-----+----+-----------+--------------------+---+-----+--------+-----+-----+----+-----+
|title|city|postal_code|real estate provider|url|offer|propType|beadR|bathR|sqft|price|
+-----+----+-----------+--------------------+---+-----+--------+-----+-----+----+-----+
+-----+----+-----------+--------------------+---+-----+--------+-----+-----+----+-----+



## Question 10 (Filter by type of property)

In [32]:
df_sale = df_sale.filter(df_sale.propType == "house")
df_sale.show(10)

+--------------+------------+-----------+--------------------+--------------------+-----+--------+-----+-----+----+-------+
|         title|        city|postal_code|real estate provider|                 url|offer|propType|beadR|bathR|sqft|  price|
+--------------+------------+-----------+--------------------+--------------------+-----+--------+-----+-----+----+-------+
|House for sale|      Boston|      02118|Campion & Company...|https://www.zillo...| sale|   house|    4|  7.0|6836|9950000|
|House for sale| East Boston|      02128|Berkshire Hathawa...|https://www.zillo...| sale|   house|    3|  3.0|2313| 899000|
|House for sale|      Boston|      02113|       CL Properties|https://www.zillo...| sale|   house|    2|  1.0|1165|1200000|
|House for sale|      Boston|      02129|All Star Realty, ...|https://www.zillo...| sale|   house|    3|  4.0|1680|1119000|
|House for sale|South Boston|      02127|             Compass|https://www.zillo...| sale|   house|    4|  3.0|2043|1699000|
|House f

In [33]:
def check_pType(propType):
  return (propType != "house") 

check_pType_udf = udf(check_pType, BooleanType())
test = df_sale.filter(check_pType_udf(df_sale.propType))

test.show()

+-----+----+-----------+--------------------+---+-----+--------+-----+-----+----+-----+
|title|city|postal_code|real estate provider|url|offer|propType|beadR|bathR|sqft|price|
+-----+----+-----------+--------------------+---+-----+--------+-----+-----+----+-----+
+-----+----+-----------+--------------------+---+-----+--------+-----+-----+----+-----+



## Question 10 (Calculating $$$/sqft)

In [35]:
for i in range(10):
  cat=df_sale.filter(df_sale.beadR == i)
  sums=cat.groupBy().sum().collect()

  if (i==1):
    cost = "no data"
    print("Monetary cost per square feet for houses with",i,"beadrooms:" , cost)
  else:
    cost = int(sums[0][3]/sums[0][2])
    
    print("Monetary cost per square feet for houses with",i,"beadrooms:" , cost,"$")
  print()

Monetary cost per square feet for houses with 0 beadrooms: 1250 $

Monetary cost per square feet for houses with 1 beadrooms: no data

Monetary cost per square feet for houses with 2 beadrooms: 714 $

Monetary cost per square feet for houses with 3 beadrooms: 698 $

Monetary cost per square feet for houses with 4 beadrooms: 1125 $

Monetary cost per square feet for houses with 5 beadrooms: 906 $

Monetary cost per square feet for houses with 6 beadrooms: 529 $

Monetary cost per square feet for houses with 7 beadrooms: 1126 $

Monetary cost per square feet for houses with 8 beadrooms: 1567 $

Monetary cost per square feet for houses with 9 beadrooms: 1108 $



In [36]:
def check_beads(beadR):
  return beadR == 10

check_beads_udf = udf(check_beads, BooleanType())
test = df_sale.filter(check_beads_udf(df_sale.beadR))

test.show()

+-----+----+-----------+--------------------+---+-----+--------+-----+-----+----+-----+
|title|city|postal_code|real estate provider|url|offer|propType|beadR|bathR|sqft|price|
+-----+----+-----------+--------------------+---+-----+--------+-----+-----+----+-----+
+-----+----+-----------+--------------------+---+-----+--------+-----+-----+----+-----+



There are no houses that have follow all resctrictions and have 10 beads even if we do not filter them out directly

In [34]:
df_sale.filter(df_sale.beadR == 9).show(99)


+--------------+------+-----------+--------------------+--------------------+-----+--------+-----+-----+----+-------+
|         title|  city|postal_code|real estate provider|                 url|offer|propType|beadR|bathR|sqft|  price|
+--------------+------+-----------+--------------------+--------------------+-----+--------+-----+-----+----+-------+
|House for sale|Boston|      02108|Campion & Company...|https://www.zillo...| sale|   house|    9|  7.0|8979|9950000|
|House for sale|Boston|      02108|Campion & Company...|https://www.zillo...| sale|   house|    9|  7.0|8979|9950000|
|House for sale|Boston|      02108|Campion & Company...|https://www.zillo...| sale|   house|    9|  7.0|8979|9950000|
|House for sale|Boston|      02108|Campion & Company...|https://www.zillo...| sale|   house|    9|  7.0|8979|9950000|
|House for sale|Boston|      02108|Campion & Company...|https://www.zillo...| sale|   house|    9|  7.0|8979|9950000|
|House for sale|Boston|      02108|Campion & Company...|

Multiple copies can lead to irrational results. For example 0 beads to be more expensive per sqft than 2 beads
or going from 6 to 8 to triple the value per sqft or going for 6 to 5 to half the cost.